### Task 3: Knowledge Based Agents
The test results are in the "TestResults" folder. Since the output didn't fit on a screenshot, I saved the output as .txt files.

In [4]:
import gym
import fh_ac_ai_gym

In [ ]:
class KnowledgeBase_CNF:
    def __init__(self):
        self.facts = set()
        self.cnf_clauses = []

    def tell(self, sentence):
        clause = frozenset(sentence)
        if clause in self.cnf_clauses:
            #print(f"[Info] Klausel bereits vorhanden: {clause}")
            pass
        else:
            contradictions = [existing for existing in self.cnf_clauses if self._contradicts_existing(clause, existing)]
            if contradictions:
                print(f"[Warnung] Widerspruch erkannt: {clause} widerspricht {contradictions}")
                return
            self.cnf_clauses.append(clause)
            #print(f"[Info] Neue Klausel hinzugefügt: {clause}")

    def _contradicts_existing(self, clause1, clause2):
        for literal in clause1:
            negation = f"-{literal}" if literal[0] != "-" else literal[1:]
            if negation in clause2:
                new_clause = frozenset(l for l in clause1 | clause2 if l != literal and l != negation)
                if not new_clause:
                    return True
        return False

    def ask(self, query):
        return self._resolution_ask(query)

    def _resolution_ask(self, query):
        negation = f"-{query}" if query[0] != "-" else query[1:]
        if frozenset({negation}) in self.cnf_clauses:
            return False
        if frozenset({query}) in self.cnf_clauses:
            return True

        clauses = self.cnf_clauses.copy()
        query_clause = frozenset({negation})
        clauses.append(query_clause)

        seen = set(map(frozenset, clauses))

        while True:
            new_clauses = []
            for i in range(len(clauses)):
                for j in range(i + 1, len(clauses)):
                    if query in clauses[i] or negation in clauses[i] or query in clauses[j] or negation in clauses[j]:
                        resolvent = self._resolve(clauses[i], clauses[j])
                        if resolvent == frozenset():
                            return True
                        if resolvent and frozenset(resolvent) not in seen:
                            new_clauses.append(resolvent)
                            seen.add(frozenset(resolvent))
            if not new_clauses:
                return False
            clauses.extend(new_clauses)
            
            
    def _resolve(self, clause1, clause2):
        for literal in clause1:
            negation = f"-{literal}" if literal[0] != "-" else literal[1:]
            if negation in clause2:
                new_clause = frozenset(l for l in clause1 | clause2 if l != literal and l != negation)
                return new_clause
        return False

In [6]:
ACTIONS = {"walk": 0, "turn_left": 1, "turn_right": 2, "shoot": 3, "grab": 4, "climb": 5}


In [7]:
def get_neighbors(x, y):
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 4 and 0 <= ny < 4:
            neighbors.append((nx, ny))
    return neighbors

In [8]:
def update_kb_cnf(kb, obs, x, y):
    pos = f"{x + 1}{y + 1}"

    kb.tell({f"S{pos}"} if obs["stench"] else {f"-S{pos}"})
    kb.tell({f"B{pos}"} if obs["breeze"] else {f"-B{pos}"})
    kb.tell({f"G{pos}"} if obs["glitter"] else {f"-G{pos}"})

    kb.tell({f"-W{pos}"}) 
    kb.tell({f"-P{pos}"})
    
    neighbors = get_neighbors(x, y)
    neighbor_pos = [f"{nx + 1}{ny + 1}" for nx, ny in neighbors]

    if obs["stench"]:
        kb.tell(set(f"W{npos}" for npos in neighbor_pos) | {f"-S{pos}"})
    else:
        for npos in neighbor_pos:
            kb.tell({f"-W{npos}"})

    if obs["breeze"]:
        kb.tell(set(f"P{npos}" for npos in neighbor_pos) | {f"-B{pos}"})
    else:
        for npos in neighbor_pos:
            kb.tell({f"-P{npos}"})
    if obs["glitter"]:
        kb.tell({f"G{pos}"})
    else:
        kb.tell({f"-G{pos}"})

    for npos in neighbor_pos:
        if kb.ask(f"W{npos}"):
            print(f"[Sicher erkannt] Wumpus in Feld {npos}")
        if kb.ask(f"P{npos}"):
            print(f"[Sicher erkannt] Pit in Feld {npos}")

In [9]:
def move_and_update(env, kb, action, x, y, direction):
    obs, reward, done, _ = env.step(action)
    env.render()

    new_x, new_y, new_direction = x, y, direction

    if action == ACTIONS["walk"]:
        candidate_x, candidate_y = x, y
        if direction == 0:  # Osten
            candidate_x += 1
        elif direction == 1:  # Süden
            candidate_y -= 1
        elif direction == 2:  # Westen
            candidate_x -= 1
        elif direction == 3:  # Norden
            candidate_y += 1

        if 0 <= candidate_x < 4 and 0 <= candidate_y < 4:
            new_x, new_y = candidate_x, candidate_y
            print(f"Agent läuft von [{x+1},{y+1}] nach [{new_x+1},{new_y+1}]")
        else:
            print("[WARNUNG] Versuch, aus der Welt zu laufen – Koordinaten bleiben gleich.")

    elif action == ACTIONS["turn_right"]:
        new_direction = (direction + 1) % 4
    elif action == ACTIONS["turn_left"]:
        new_direction = (direction - 1) % 4

    richtung_namen = ["Ost", "Süd", "West", "Nord"]
    print(f"Ausrichtung nach: {richtung_namen[new_direction]}")

    if new_x != x or new_y != y:
        update_kb_cnf(kb, obs, new_x, new_y)

    return obs, new_x, new_y, new_direction, done, reward

In [10]:
def reset_world_and_kb(env):
    obs = env.reset()
    env.render()
    kb = KnowledgeBase_CNF()
    return obs, kb, 0, 0, 0

In [11]:
def test_1_startfeld(env):
    print("\nTest 1: Nur vom Startfeld aus – Wumpus eingrenzen\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_cnf(kb, obs, 0, 0)
    print("Geruch am Start?", "S11" in kb.facts or any("S11" in clause for clause in kb.cnf_clauses))
    print("Kann W21 gefolgert werden?", kb.ask("W21"))
    print("Kann W12 gefolgert werden?", kb.ask("W12"))
    print("Erwartung: Beides False, da unklar ob Rechts oder oben.")

In [12]:
def test_2_wumpus_bei_1_2(env):
    print("\nTest 2: Nach oben laufen (Feld [1,2]) und Wumpus suchen\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_cnf(kb, obs, x, y)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_left"], x, y, direction)
    print("Kann W21 gefolgert werden?", kb.ask("W21"))
    print("Kann W12 gefolgert werden?", kb.ask("W12"))
    print("Erwartung: Beides False, da unklar ob Rechts oder oben.")
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Kann W21 gefolgert werden?", kb.ask("W21"))
    print("Kann W12 gefolgert werden?", kb.ask("W12"))
    print("Erwartung: W21=True, W12=False")

In [10]:
def test_3_pit_bei_3_1(env):
    print("\nTest 3: Gehe zu [3,1] und schaue ob das Pit erkannt wird\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_cnf(kb, obs, x, y)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_left"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    print("Pit in [1,3]?", kb.ask("P13"))
    print("Pit in [2,2]?", kb.ask("P22"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))
    print("Erwartung: p14=False, p23=False da man sich nicht sicher sein kann")

In [11]:
def test_4_pit_herumgehen(env):
    print("\nTest 4: Gehe zu [3,1] und laufe um das Pit\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_cnf(kb, obs, x, y)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_left"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    print("Pit in [1,3]?", kb.ask("P13"))
    print("Pit in [2,2]?", kb.ask("P22"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)


    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))
    print("Erwartung: p14=False, p23=true da man sich sicher sein kann")

In [13]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_CNF()
test_1_startfeld(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 1: Nur vom Startfeld aus – Wumpus eingrenzen

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
Geruch am Start? True
Kann W21 gefolgert werden? False
Kann W12 gefolgert werden? False
Erwartung: Beides False, da unklar ob Rechts oder oben.
Not necessary since no seperate window was opened


In [14]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_CNF()
test_2_wumpus_bei_1_2(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 2: Nach oben laufen (Feld [1,2]) und Wumpus suchen

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : -1
Ausricht

In [14]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_CNF()
test_3_pit_bei_3_1(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 3: Gehe zu [3,1] und schaue ob das Pit erkannt wird

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : -1
Ausrich

In [15]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_CNF()
test_4_pit_herumgehen(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 4: Gehe zu [3,1] und laufe um das Pit

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : -1
Ausrichtung nach: Nor